# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [17]:
data['race'].value_counts()

w    2435
b    2435
Name: race, dtype: int64

In [16]:
# number of callbacks for black-sounding names
sum_b = sum(data[data.race=='b'].call)
sum_w = sum(data[data.race=='w'].call)
print("number of callbacks for black-sounding names = ", sum_b)
print("number of callbacks for white-sounding names = ", sum_w)

number of callbacks for black-sounding names =  157.0
number of callbacks for white-sounding names =  235.0


In [22]:
data[['race', 'call']].head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


In [36]:
mean_call = np.mean(data['call'])

w_call = data[data['race'] == 'w'].call
b_call = data[data['race'] == 'b'].call

w_call_shifted = w_call - np.mean(w_call) - mean_call
b_call_shifted = b_call - np.mean(b_call) - mean_call

def bootstrap_replicate(data, func):
    bs_samples = np.random.choice(data, len(data))
    return func(bs_samples)

def draw_bs_replicates(data, func, size):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate(data, func)
    return bs_replicates

bs_replicates_w = draw_bs_replicates(w_call_shifted, np.mean, 10000)
bs_replicates_b = draw_bs_replicates(b_call_shifted, np.mean, 10000)

bs_replicates = bs_replicates_w - bs_replicates_b

empirical_diff_means = np.mean(w_call) - np.mean(b_call)
print('mean of w group = ', np.mean(w_call))
print('mean of b group = ', np.mean(b_call))
print('difference of means of sample groups = ', empirical_diff_means)

p = np.sum(bs_replicates > empirical_diff_means) / len(bs_replicates)
print('p value = ',p)

conf_interval = np.percentile(bs_replicates, [2.5, 97.5])
print('95% confidence interval of difference of means = ', conf_interval)

bs_replicates_mean = np.mean(bs_replicates)
bs_replicates_std = np.std(bs_replicates)
bs_replicates_std_error = bs_replicates_std / np.sqrt(10000)

#critical value of 95% confidence interval = 1.96
margin_error = bs_replicates_std_error * 1.96
print('95% confidence interval, margin of error = ',margin_error)

mean of w group =  0.09650924056768417
mean of b group =  0.0644763857126236
difference of means of sample groups =  0.03203285485506058
p value =  0.0001
95% confidence interval of difference of means =  [-0.0147844   0.01560575]
95% confidence interval, margin error =  0.000153159802954


Statistical Analysis Report

1. race is a categorical variable, and 'w' and 'b' are equally splitted. And call is a continuous variable, but behaves more like categorical variable, since it only has two values 1.0 or 0.0. 

2. CLT still applies because the sample size n > 30. The test I am going to do is the two-sample bootstrap hypothesis test of difference of means of call variables between 'w' and 'b' groups. 

3. The null hypothesis is that the mean of 'w' group's call is equal to the mean of 'b' group's call. The alternative hypothesis is the mean of 'w' group's call is significant different than the mean of 'b' group call.

4. The close to Zero p value strongly rejects the null hypothesis and accepts the alternative hypothesis, which means there is significant difference between the mean of the number of callbacks the while people are getting and the mean of the number of callback that the black people are getting not due to the sampling errors. Based on this analysis without looking into any other features from the data sample, it appears that race makes a significant difference on the rate of callbacks of resumes.

5. Even though the null hypothesis test is pointing out the siginficant impact of race to the rate of callbacks of resumes, it is still insufficient to conclude that race is the most important factor in callback success. Because peoples' qualification also determines the callback success, which is combined with education and experience. In order to find out the key factors for the callback success, I would do further analysis of other attributes, such as education, positions, years of experience, computer skills, special skills ect.